In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import feather
import datetime

In [11]:
ls '../src'

TOTH_ACCOUNT_PROFILE.7z*  TOTH_PAYMENT_PRODUCT.7z*  df_accs_samp-raw
TOTH_ACCOUNT_PROFILE.txt  TOTH_PRODUCT.7z*          df_invs_samp-raw
TOTH_INVOICE.7z*          TOTH_PRODUCT.txt          df_pays_samp-raw
TOTH_INVOICE.txt          TOTH_SCOMS.7z*            df_prods_samp-raw
TOTH_PAYMENT.txt          TOTH_SCOMS.txt            df_scoms_samp-raw


In [12]:
PATH  = '../src/'

# start from service complaints
df_scoms = pd.read_csv(f'{PATH}TOTH_SCOMS.txt',encoding = "cp874",nrows=100_000)
df_scoms = df_scoms.loc[df_scoms.SERVICE_TYPE2.isin(['ADSL','FTTx'])]

df_scoms['FAILDATE'] = pd.to_datetime(df_scoms['FAILDATE'],format = '%d/%m/%Y %H:%M:%S.%f')
df_scoms['COMDATE'] = pd.to_datetime(df_scoms['COMDATE'],format = '%d/%m/%Y %H:%M:%S.%f',errors = 'ignore')
df_scoms['CHKWORK'] = [c[:2] for c in df_scoms.CHKWORK.fillna('NA')]
df_scoms = pd.get_dummies(df_scoms, columns=['CHKWORK','SERVICE_TYPE2','FAULTRANGE'], dummy_na=True)
# drop the null complete date
df_scoms.dropna(subset=['COMDATE'],inplace=True)
df_scoms.head()

In [27]:
tel_list = df_scoms.TEL.unique()

# product
df_prods = pd.read_csv(f'{PATH}TOTH_PRODUCT.txt')
df_prods = df_prods.loc[df_prods.TEL.isin(tel_list)]
df_prods['TEL_START_DT'] = pd.to_datetime(df_prods['TEL_START_DT'],format='%m/%d/%Y')
df_prods['TEL_END_DT'] = pd.to_datetime(df_prods['TEL_END_DT'],format='%m/%d/%Y',errors='ignore')

df_prods = df_prods.loc[df_prods.CHARGE > 100]

df_prods.head()

,ACCOUNT,TEL,TEL_START_DT,TEL_END_DT,TARIFF,QUANTITY,CHARGE
289,B2A9B9930AAB063D7B6F88FBBCC4C40D,50DC72E95344D07C0C095422F482A067,2018-09-06,2021-09-05,Plan 5354,1,450.0
613,B2A9B9930AAB063D7B6F88FBBCC4C40D,50DC72E95344D07C0C095422F482A067,2014-09-06,2015-09-05,Plan 5045,1,1000.0
619,B2A9B9930AAB063D7B6F88FBBCC4C40D,50DC72E95344D07C0C095422F482A067,2017-05-06,2018-05-05,Plan 3107,1,499.0
772,11905624254857E8415B2323649CA0B5,4F5652AB84E2A3D58744A7E329D98D80,2014-03-24,2015-05-12,Plan 918,1,300.0
775,D241D5652995AE1ABDB246B39ECDF150,2318C0E35644BED1CB91DA51DF623E2A,2011-10-18,2011-12-15,Plan 3053,1,390.0


In [15]:
df_prods.describe()

,QUANTITY,CHARGE
count,23655.000000,23655.000000
mean,1.000803,535.948268
std,0.123536,297.698457
min,1.000000,150.000000
25%,1.000000,400.000000
50%,1.000000,590.000000
75%,1.000000,590.000000
max,20.000000,8500.000000


In [31]:
# account
acc_lst = df_prods.ACCOUNT.unique()

df_accs = pd.read_csv(f'{PATH}TOTH_ACCOUNT_PROFILE.txt',encoding = "cp874",usecols = ['ACCOUNT','START_DT','BILL_DELIVER','PAY_METHOD','PROVINCE'])
df_accs = df_accs.loc[df_accs.ACCOUNT.isin(acc_lst)]
df_accs['START_DT'] = pd.to_datetime(df_accs['START_DT'],format='%m/%d/%Y')

print('datetime range: ',df_accs.START_DT.min(),' - ', df_accs.START_DT.max())

datetime range:  1970-01-01 00:00:00  -  2017-08-16 00:00:00


In [ ]:
# invoice
usecols=['ACCOUNT','DUE_DATE','SETTLED_DT','TOTAL','PAYMENT','OUTSTANDING']
df_invs = []
chunk_size = 1e6
for chunk in pd.read_csv(f'{PATH}TOTH_INVOICE.txt', encoding = "cp874", chunksize=chunk_size):    
    chunk = chunk.loc[chunk.ACCOUNT.isin(acc_lst)]
    df_invs.append(chunk)
df_invs = pd.concat(df_invs)
del chunk

In [ ]:
df_invs.head()

In [22]:
df_accs.reset_index(drop=True,inplace=True)
df_invs.reset_index(drop=True,inplace=True)
df_prods.reset_index(drop=True,inplace=True)
df_scoms.reset_index(drop=True,inplace=True)

df_accs.to_feather(f'{PATH}df_accs_samp-raw')
df_invs.to_feather(f'{PATH}df_invs_samp-raw')
df_prods.to_feather(f'{PATH}df_prods_samp-raw')
df_scoms.to_feather(f'{PATH}df_scoms_samp-raw')

In [1]:
#**********************************************************************

In [14]:
# LEAVE PAYMENT FOR NOW...

inv_lst = df_invs.INVOICE.unique()

# payment
df_pays = []
chunk_size = 1e6
for chk in pd.read_csv(f'{PATH}TOTH_PAYMENT.txt', encoding = 'cp874', chunksize= chunk_size):
    df_pays.append(chk.loc[chk.INVOICE.isin(inv_lst)])
df_pays = pd.concat(df_pays)

df_pays.reset_index(drop=True,inplace=True)
df_pays.to_feather(f'{PATH}df_pays_samp-raw')

In [ ]:
# memory is full when trying to join 2 dfs... try move the work to gcp later on

In [ ]:
df_inv_x_p = df_invs.merge(df_pays,how='inner',on='INVOICE',suffixes=('_l','_r'))

In [ ]:
# check total row first
print(df_invs.shape, df_inv_x_p.shape)

In [11]:
# ---> 1 invoice can have more than one payment method... but is it automatically generated from the payment system

# check df_pays
df_pays['tran_count'] = df_pays.groupby('INVOICE').transform('count').iloc[:,0]

In [12]:
df_pays.loc[df_pays.tran_count >1]

,INVOICE,PAY_DT,PAY_METH,PAY_AMT,tran_count
0,D09EA2DE95B10FD768255CF5AD487A1B,1/30/2018,3 Teresa,10.18,2
1,922E7DD242E4620AC725016D8F00BD6D,12/19/2017,Cash,117.60,2
2,922E7DD242E4620AC725016D8F00BD6D,12/19/2017,3 Teresa,5.45,2
3,67DEC66B21A4EA345778530A6A50DB8E,11/7/2017,Cash,102.00,2
4,67DEC66B21A4EA345778530A6A50DB8E,11/7/2017,3 Teresa,5.00,2
6,99A55DD38BF504C5D7EC3A52B0B43EC0,10/3/2017,3 Teresa,11.26,2
7,99A55DD38BF504C5D7EC3A52B0B43EC0,10/3/2017,Cash,115.00,2
11,53D0F205A1A1462F79BA8AF90AE945C9,3/29/2017,Cash,137.29,2
12,53D0F205A1A1462F79BA8AF90AE945C9,3/29/2017,3 Teresa,11.44,2
26,EED83C811627C09B80958427B02CE367,1/15/2019,Cash,113.42,2


In [13]:
# check how pay amount from payment is related to payment in invoice
df_inv_x_p.head(20)

,ACCOUNT,INVOICE,BILL_DT,DUE_DATE,SETTLED_DT,AMT,VAT,TOTAL,ADJUST,PAYMENT,OUTSTANDING,PAY_DT,PAY_METH,PAY_AMT
0,B2A9B9930AAB063D7B6F88FBBCC4C40D,E645402650BF5ABF19140B45873986DB,JUN-14,7/7/2014,7/31/2014,1763.50,123.45,1886.95,NaN,1886.95,0.0,NaN,NaN,NaN
1,B2A9B9930AAB063D7B6F88FBBCC4C40D,1A3BA86FCA850BA121845F21D8683A35,AUG-14,9/7/2014,9/25/2014,1556.83,108.98,1665.81,NaN,1665.81,0.0,NaN,NaN,NaN
2,B2A9B9930AAB063D7B6F88FBBCC4C40D,DFA66608782FCB4CD887843B0755C7B9,SEP-14,10/7/2014,9/25/2014,1866.00,130.62,1996.62,NaN,1996.62,0.0,NaN,NaN,NaN
3,B2A9B9930AAB063D7B6F88FBBCC4C40D,25F3855660E98F52855318DA06761EAA,OCT-14,11/7/2014,11/3/2014,1650.00,115.50,1765.50,NaN,1765.50,0.0,NaN,NaN,NaN
4,B2A9B9930AAB063D7B6F88FBBCC4C40D,6D27F1EF6E5CFC14DF69BBA6AB305131,DEC-14,1/7/2015,1/6/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0,NaN,NaN,NaN
5,B2A9B9930AAB063D7B6F88FBBCC4C40D,732F660AF93683CF9F691CF8E84F70FC,JAN-15,2/7/2015,3/10/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0,NaN,NaN,NaN
6,B2A9B9930AAB063D7B6F88FBBCC4C40D,7592E5968907DBEADFF826A081B4E2EB,FEB-15,3/7/2015,3/10/2015,1856.00,129.92,1985.92,NaN,1985.92,0.0,NaN,NaN,NaN
7,B2A9B9930AAB063D7B6F88FBBCC4C40D,1278620885FBC93EA048544DC76F1A98,MAR-15,4/7/2015,5/14/2015,1850.00,129.50,1979.50,NaN,1979.50,0.0,NaN,NaN,NaN
8,B2A9B9930AAB063D7B6F88FBBCC4C40D,B225572D1E37AE77D698961D0EDAB879,MAY-15,6/7/2015,7/14/2015,1650.00,115.50,1765.50,NaN,1765.50,0.0,NaN,NaN,NaN
9,B2A9B9930AAB063D7B6F88FBBCC4C40D,2ED1AE3B6521FAB14AB77A63865000C0,JUL-15,8/7/2015,9/14/2015,1650.00,115.50,1765.50,NaN,1765.50,0.0,NaN,NaN,NaN
